# Thinking in tensors, writing in PyTorch

A hands-on course by [Piotr Migdał](https://p.migdal.pl) (2019).
This notebook prepared by [Weronika Ormaniec](https://github.com/werkaaa) and Piotr Migdał.

## Notebook 3: Linear regression

[Linear regression](https://en.wikipedia.org/wiki/Linear_regression) is one of the most common predictive models. In plain words, we fit a straight line that fits to the data. Mathematically speaking, we use linear combination of input variables to predict the output variable.

$$y = a_1 x_1 + \ldots a_n x_n + b$$

Before moving any further, try to experience it viscerally with [Ordinary Least Squares Regression
Explained Visually - Visually Explained](http://setosa.io/ev/ordinary-least-squares-regression/):

![http://setosa.io/ev/ordinary-least-squares-regression/](imgs/linreg_setosa.png)

However, it occurs that lots of dependencies in the actual world can be described just by fitting a linear equation to the observed data. That's what we are going to do now!

In Python we typically use [LinearRegression from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Here we use PyTorch to show everything step-by-step. Moreover, linear regression is a building block of any regression with deep learning - so it is good to understand it well!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from livelossplot import PlotLosses
from ipywidgets import interact, fixed

## Data

Have you ever wondered what is the relation between brain and body weights among various animal species?

Let's try a [Brain to Body Weight Dataset](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_Brain2BodyWeight)!

> These data represent the relation between weights of the body and brain of various species. It may be used to discuss bivariate exploratory and quantitative data analyses in the case of allometric relationships. Brain-to-body weight ratio is assumed to be related to species intelligence. The encephalization quotient is a more complex measurement that takes into account allometric effects of widely divergent body sizes across several taxa. The brain-to-body mass ratio is a simpler measure of encephalization within species.

In [ ]:
data = pd.read_csv("data/Animals.csv", index_col='Species')
data

In [ ]:
# Let's make a scatter plot
data.plot.scatter(x="BodyWeight(kg)", y="BrainWeight(kg)")

At first glance it does not resemble any particular dependance. However, if we change the scale something interesting can be spotted with [logarithmic scaling](https://simple.wikipedia.org/wiki/Logarithmic_scale):

In [ ]:
data.plot.scatter(x="BodyWeight(kg)", y="BrainWeight(kg)", logx=True, logy=True);

We see a clear dependence that the bigger body weight (on average) the bigger brain weight.
Let's investigate that!

### A toy example

**TODO: This one or just move with the original numbers, otherwise we lose the flow**

Let's consider two sets of numbers:

In [ ]:
x = np.array([1., 2., 3., 4.])
y = np.array([3.2, 5.1, 6.9, 9.3])

plt.scatter(x, y)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

On the scatterplot it can be easly seen that the relationship between presented data is almost linear. We will try to apply the equation:

$$ y = ax+b$$

to the analysed dataset. The only problem is how to find $a$ and $b$.

Try to find a proper line manually!

In [ ]:
def plot_model(a, b, x, y):
    y_pred = a * x + b
    plt.plot(x, y_pred, 'g')
    plt.scatter(x, y)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.ylim([-1, 12])
    plt.show()
    
interact(plot_model, 
         a=(-1, 5), 
         b=(-0.5, 4),
         x=fixed(x),
         y=fixed(y)
        );

### Loss function

We will try to somehow measure if the coefficients in the equation are good enough to describe our problem. In order to do it we will define a loss function - an equation that will tell us how much our approximation differs from the expected output. 

The loss function should:

* depend only on the coefficients of the model, expected output and our approximation,
* shrink if our approximation is becoming better and grow if it gets worse.

When it comes to linear regression the most common approach is the least-squares loss function. We will calculate the average square of the vertical deviations from each data point to the line. Since we first square the dviations, it does not matter if the data point is above or below the line. 


$$y^{ pred}_{i} = ax_{i}+b $$
$$L=\frac{1}{N}\sum_{i=1}^N( y^{pred}_{i} - y_{i})^2 $$

We will get random $a$ and $b$ and apply our loss function.

In [ ]:
aa_ = np.linspace(0, 4, num=100)
bb_ = np.linspace(-4, 8, num=100)
aa, bb = np.meshgrid(aa_, bb_)

def loss_numpy(aa, bb, x, y):
    loss = np.zeros_like(aa)
    for i in range(len(loss)):
        for j in range(len(loss[0])):
            loss[i][j] = ((aa[i,j] * x + bb[i,j] - y)**2).sum()
    return loss

cs = plt.contour(aa, bb, np.sqrt(loss_numpy(aa, bb, x, y)), cmap='coolwarm')
plt.clabel(cs, inline=1, fontsize=10)
plt.title("Loss")
plt.xlabel('a')
plt.ylabel('b')

In [ ]:
W = torch.randn(1,2)
X = torch.tensor([[1.0,1.0,1.0,1.0], x]).float()  # NO, we want a and b separatly, without such tricks!
Y = torch.tensor(y).float()

# TO FIX: No consequti
print(W)
print(X)
print(Y)

In [ ]:
def Y_pred(W, X):
     return W.matmul(X) # matrix multiplication

Y_pred(W, X)

In [ ]:
# TODO: only loss, without prediction 
# TODO: loss LOWERCASE
def Loss(W, X, Y):
    y_pred = Y_pred(W, X)
    return (y_pred - Y).pow(2).mean()
Loss(W, X, Y)

### Minimizing loss function

**TODO: Write that for lin reg there is an explicit formula, and write so.**

You will find more detailed explanation of what will happen here in the chapter "Gradient descent".

Since we have defined the loss function, we should minimize it. By doing so we will step by step rotate and move the line, so it will reflect the actual location of data points. In order to do it we need to repeatedly shift the weights till we find a minimum of the loss function. What we need is a mathematical operation that will tell us how the loss function will change, if we increase or decrease $a$ and $b$. The operation we are looking for is partial derivative.

$$\dfrac{\partial L}{\partial a}  = \frac{2}{N}\sum_{i=0}^N (y^{pred}_{i} -y_{i}) \cdot x_{i}$$ 

$$\dfrac{\partial L}{\partial b} = \frac{2}{N}\sum_{i=0}^N (y^{pred}_{i} -y_{i})$$

In [ ]:
def dL_da(W, X, Y):
    y_pred = Y_pred(W, X)
    return 2*((y_pred - Y)*X[1]).mean()

dL_da(W, X, Y)

In [ ]:
def dL_db(W, X, Y):
    y_pred = Y_pred(W, X)
    return 2*(y_pred - Y).mean()

dL_db(W, X, Y)

Two more things we have to specify is **learning\_rate** - hyperparamiter that will define how much the value of the derivative will influance the change of $a$ and $b$ and **num\_epochs** - hyperparameter defining how many iterations it will take to sufficiently minimiaze the loss function.

In [ ]:
# TODO: slightly too much abstraction nonsese
# it IMHO it shoudn't be a separe function
def gradient_step(W, X, Y, learnig_rate):
    W[0][0] -= learnig_rate * dL_db(W, X, Y)
    W[0][1] -= learnig_rate * dL_da(W, X, Y)
    return W

In [ ]:
def minimise_loss_function(W, X, Y, learning_rate, num_epochs):
    loss_history = []
    for i in range(num_epochs):
        W = gradient_step(W, X, Y, learning_rate)
        loss_history.append(Loss(W, X, Y))
    return W, loss_history

In [ ]:
print(W)  # TO FIX: 

learning_rate = 0.1
num_epochs = 30
W_trained, loss_history = minimise_loss_function(W, X, Y, learning_rate, num_epochs)

print(W_trained, loss_history)  # TO FIX: Please. no.
plt.plot(list(range(num_epochs)), loss_history)  # use livelossplot instead

In [ ]:
b = W.numpy()[0][0]
a = W.numpy()[0][1]
plot_model(b, a, np.array(x), y)

And that is how we find the propper line!

### Linear regression using PyTorch

Knowing how linear regression works, let's come back to the relation between body and brain weights. This time we will use built-in PyTorch functions.

Firstly, we need to prepare the data. 

**TO DO: How? Explain steps in plain English.**

In [ ]:
X = torch.tensor(np.log(data['BodyWeight(kg)']))
Y = torch.tensor(np.log(data['BrainWeight(kg)']))
X = X.view(1, 27, 1)  # TOFIX: never use hardcoded sample size
Y = Y.view(1, 27, 1)

Instead of initializing the coefficients manually, we can define the model using a built in class. Since both input and output in the analysed problem have only one dimension we set **(1,1)** as arguments of **nn.Linear**.

**TODO: Nope. It's an abstraction rocket jump. Plus: changing a dataset AND method in no good.** 

In [ ]:
model = nn.Linear(1, 1)
print(model.weight)
print(model.bias)

Insted of **gradient\_step** function, we will define an **optimizer** with learning rate and built-int **loss function**.

In [ ]:
# TO FIX: one block of code please
optim = torch.optim.SGD(model.parameters(), lr=0.03)

In [ ]:
loss_function = F.mse_loss
loss = loss_function(model(X), Y)
print(loss)

Before training the model, let's see what does the line with random cefficients look like.

In [ ]:
def plot_model_annotate(b, a, x, y, ann):
    # TO FIX: it mixes objective and global matplotlib API
    fig, ax = plt.subplots()
    ax.scatter(x, y)
    y_pred = b + a * x 
    ax.plot(x, y_pred, 'g')
    plt.xlabel("BodyWeight(kg)")
    plt.ylabel("BrainWeight(kg)")
    
    for i, txt in enumerate(ann):
        if i%5 == 1: 
            ax.annotate(txt, (X_viewed.numpy()[i], Y_viewed[i]))
    plt.show()
            
    print('a =', a)
    print('b =', b)
    
X_viewed = X.view(27)  # remove those two lines
Y_viewed = Y.view(27)
plot_model_annotate(model.bias.item(), model.weight.item(), X_viewed.numpy(), Y_viewed.numpy(), data.index)

Now we can train the model - minimise the loss function.

In [ ]:

def train(num_epochs, X, Y, model, loss_function, optim):
    loss_history = []
    preds = torch.tensor([])
    liveloss = PlotLosses()

    for epoch in range(num_epochs):
        
        epoch_loss = 0.0
        
        Y_pred = model(X)
        loss = loss_function(Y_pred, Y)
        
        loss.backward()
        optim.step()
        optim.zero_grad()
        
        preds = torch.cat([preds, Y_pred], 0)
        
        epoch_loss = loss.data.item()
        
        avg_loss = epoch_loss / len(X)

        liveloss.update({
            'loss': avg_loss,
            'a': model.weight[0][0].item(),
            'b': model.bias[0].item()
        })
        liveloss.draw()

    return preds

predictions = train(50, X, Y, model, loss_function, optim)


We can see, how the line was changing during learning process:

In [ ]:
X_viewed = X.view(27)
predictions = predictions.view([-1, 27])
plt.scatter(X, Y)
for i in range(predictions.size()[0]):
    if i%10 == 0:
        plt.plot(X_viewed.numpy(),predictions[i].detach().numpy())
    

Let's see if we fitted the final line properly!

In [ ]:
X_viewed = X.view(27)
Y_viewed = Y.view(27)
plot_model_annotate(model.bias.item(), model.weight.item(), X_viewed.numpy(), Y_viewed.numpy(), data.index)

It fits the data much better than at the begining. We have found the relation between brain and body weights among various animal species.

In [ ]:
predictions_n = predictions[-1].exp().detach().numpy()
df = pd.DataFrame({'Species'  : data.index, 'PredictedBrainWeight(kg)': predictions_n})
df.index = df["Species"]

In [ ]:
# TO FIX: no, no concat; just ad a column
data_p = pd.concat([data, df['PredictedBrainWeight(kg)']], axis=1)
data_p

We can now compare predicted brain weights with actual data. What does it mean that the actual brain weight is bigger than predicted one? Is an animal more clever in that case?

## Homework

If you want to practice linear regression, here is another dataset. It describes the relation between weight and average heart rate of various animals. 

**TODO: Which dataset? Don't use `data2` or anything similar as a name.**

(Tip: try to scale the data, by taking logarithm of both values)

In [ ]:
data2 = pd.read_csv("data/Heart_rate_and_weight.csv", index_col=0)
data2

## Extra

Here are some interesting websites on the subject of linear regression:


* [Linear regression](http://www.stat.yale.edu/Courses/1997-98/101/linreg.htm)
* [Ordinary Least Squares Regression-Explained Visually](http://setosa.io/ev/ordinary-least-squares-regression/)
* [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)


Beware, even if there is some correlation, it may be not that sound:

![https://imgs.xkcd.com/comics/linear_regression.png](https://imgs.xkcd.com/comics/linear_regression.png)

![https://imgs.xkcd.com/comics/extrapolating.png](https://imgs.xkcd.com/comics/extrapolating.png)



## TODO

* Linear regression with more variables
* Consequence with upper and lower indices